# LeNet-5 Evaluation on Fashion-MNIST

Classic LeNet-5 architecture (1998) for baseline comparison:
- 2 convolutional layers with average pooling
- 3 fully connected layers
- ~60K parameters

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

batch_size = 64
num_classes = 10
learning_rate = 0.001

num_epochs = 20

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

In [ ]:
all_transforms = transforms.Compose([transforms.Resize((32,32)),
                                     transforms.ToTensor(),
                                     transforms.Normalize(mean=[0.5],
                                                          std=[0.5])
                                     ])
train_dataset = torchvision.datasets.FashionMNIST(root = './data',
                                             train = True,
                                             transform = all_transforms,
                                             download = True)

test_dataset = torchvision.datasets.FashionMNIST(root = './data',
                                            train = False,
                                            transform = all_transforms,
                                            download=True)

train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                           batch_size = batch_size,
                                           shuffle = True)


test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                           batch_size = batch_size,
                                           shuffle = True)

In [ ]:
class LeNet5(nn.Module):
    """Classic LeNet-5 architecture from 1998"""
    def __init__(self, num_classes):
        super(LeNet5, self).__init__()
        # First convolutional block
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.avgpool1 = nn.AvgPool2d(kernel_size=2, stride=2)
        
        # Second convolutional block
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5)
        self.avgpool2 = nn.AvgPool2d(kernel_size=2, stride=2)
        
        # Fully connected layers
        # Input: 32x32 -> conv1: 28x28 -> pool: 14x14 -> conv2: 10x10 -> pool: 5x5
        # Flattened size: 16 * 5 * 5 = 400
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, num_classes)
    
    def forward(self, x):
        # First conv block with tanh activation (original LeNet used tanh)
        out = torch.tanh(self.conv1(x))
        out = self.avgpool1(out)
        
        # Second conv block
        out = torch.tanh(self.conv2(out))
        out = self.avgpool2(out)
        
        # Flatten
        out = out.reshape(out.size(0), -1)
        
        # Fully connected layers
        out = torch.tanh(self.fc1(out))
        out = torch.tanh(self.fc2(out))
        out = self.fc3(out)
        return out

In [ ]:
model = LeNet5(num_classes)
model = model.to(device)

# Count parameters
total_params = sum(p.numel() for p in model.parameters())
print(f'Total parameters: {total_params:,}')

criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay = 0.005, momentum = 0.9)  

total_step = len(train_loader)

In [ ]:
# Initial training run to test the model
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):  
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

In [ ]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    print('Accuracy of the network on the {} test images: {:.2f} %'.format(10000, 100 * correct / total))

## Grid Search for Hyperparameter Tuning

Testing different combinations of:
- Optimizers: SGD, Adam
- Learning rates: 0.001, 0.01, 0.1
- Weight decay: 0, 0.005

Total configurations: 2 × 3 × 2 = 12

In [ ]:
# Grid search hyperparameters
optimizers_to_test = ['sgd', 'adam']
learning_rates = [0.001, 0.01, 0.1]
weight_decays = [0, 0.005]

# Store results
results = []

print(f"Total configurations to test: {len(optimizers_to_test) * len(learning_rates) * len(weight_decays)}")

In [ ]:
import time

config_num = 0
for optimizer_name in optimizers_to_test:
    for lr in learning_rates:
        for wd in weight_decays:
            config_num += 1
            print(f"\n{'='*60}")
            print(f"Configuration {config_num}/12")
            print(f"Optimizer: {optimizer_name}, LR: {lr}, Weight Decay: {wd}")
            print(f"{'='*60}")
            
            # Create fresh model
            model = LeNet5(num_classes).to(device)
            
            # Create optimizer based on type
            if optimizer_name == 'sgd':
                optimizer = torch.optim.SGD(model.parameters(), lr=lr, weight_decay=wd, momentum=0.9)
            else:  # adam
                optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=wd)
            
            criterion = nn.CrossEntropyLoss()
            
            # Training
            start_time = time.time()
            for epoch in range(num_epochs):
                model.train()
                for i, (images, labels) in enumerate(train_loader):
                    images = images.to(device)
                    labels = labels.to(device)
                    
                    outputs = model(images)
                    loss = criterion(outputs, labels)
                    
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()
                
                if (epoch + 1) % 5 == 0:  # Print every 5 epochs
                    print(f'  Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
            
            training_time = time.time() - start_time
            
            # Evaluation on test set
            model.eval()
            with torch.no_grad():
                correct = 0
                total = 0
                for images, labels in test_loader:
                    images = images.to(device)
                    labels = labels.to(device)
                    outputs = model(images)
                    _, predicted = torch.max(outputs.data, 1)
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()
                
                test_accuracy = 100 * correct / total
            
            print(f"  Test Accuracy: {test_accuracy:.2f}%")
            print(f"  Training Time: {training_time:.2f}s")
            
            # Store results
            results.append({
                'optimizer': optimizer_name,
                'learning_rate': lr,
                'weight_decay': wd,
                'test_accuracy': test_accuracy,
                'training_time': training_time
            })

print(f"\n{'='*60}")
print("Grid search completed!")
print(f"{'='*60}")

In [ ]:
# Display all results sorted by test accuracy
print("\n" + "="*80)
print("GRID SEARCH RESULTS - Sorted by Test Accuracy")
print("="*80)
print(f"{'Rank':<6} {'Optimizer':<10} {'LR':<10} {'Weight Decay':<13} {'Test Acc':<12} {'Time (s)':<10}")
print("-"*80)

# Sort results by test accuracy (descending)
sorted_results = sorted(results, key=lambda x: x['test_accuracy'], reverse=True)

for rank, result in enumerate(sorted_results, 1):
    print(f"{rank:<6} {result['optimizer']:<10} {result['learning_rate']:<10} "
          f"{result['weight_decay']:<13} {result['test_accuracy']:<12.2f} {result['training_time']:<10.2f}")

print("="*80)
print("\nBEST CONFIGURATION:")
best = sorted_results[0]
print(f"  Optimizer: {best['optimizer']}")
print(f"  Learning Rate: {best['learning_rate']}")
print(f"  Weight Decay: {best['weight_decay']}")
print(f"  Test Accuracy: {best['test_accuracy']:.2f}%")
print(f"  Training Time: {best['training_time']:.2f}s")
print("="*80)

## Retrain Best Configuration and Save Model

Retraining the best configuration and saving the model for later use.

In [ ]:
# Best configuration from grid search
best_optimizer = best['optimizer']
best_lr = best['learning_rate']
best_wd = best['weight_decay']

print("Training with best configuration...")
print(f"Optimizer: {best_optimizer}, LR: {best_lr}, Weight Decay: {best_wd}\n")

# Create fresh model
best_model = LeNet5(num_classes).to(device)

# Create optimizer
if best_optimizer == 'sgd':
    optimizer = torch.optim.SGD(best_model.parameters(), lr=best_lr, weight_decay=best_wd, momentum=0.9)
else:  # adam
    optimizer = torch.optim.Adam(best_model.parameters(), lr=best_lr, weight_decay=best_wd)

criterion = nn.CrossEntropyLoss()

# Training
for epoch in range(num_epochs):
    best_model.train()
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = best_model(images)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    if (epoch + 1) % 5 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Save the best model
torch.save(best_model.state_dict(), 'best_lenet5_model.pth')
print(f"\nModel saved to 'best_lenet5_model.pth'")

# Final evaluation
best_model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = best_model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    final_accuracy = 100 * correct / total
    print(f'Final Test Accuracy: {final_accuracy:.2f}%')

## Comparison Notes

**LeNet-5 (1998)** vs **Custom CNN (2020s)**

Architecture differences:
- LeNet: 2 conv layers with AvgPool, tanh activation, ~60K params
- Custom CNN: 4 conv layers with MaxPool, ReLU activation, ~1M params

Expected results:
- LeNet-5: ~87-89% accuracy (classic baseline)
- Custom CNN: ~90%+ accuracy (modern improvements)

This demonstrates the impact of architectural evolution over 25+ years.